In [4]:
import pandas as pd

In [5]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data (1).csv


In [6]:
df = pd.read_csv("data.csv")
df

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1
...,...,...,...,...
4004,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0
4005,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0
4006,https://www.activistpost.com/2017/09/ron-paul-...,"Ron Paul on Trump, Anarchism & the AltRight",NaN,0
4007,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,1


In [7]:
print(df.info())
print('-------------------------------------------------------')
print(df.isna().sum())
print('-------------------------------------------------------')
print(df['Label'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   URLs      4009 non-null   object
 1   Headline  4009 non-null   object
 2   Body      3988 non-null   object
 3   Label     4009 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 125.4+ KB
None
-------------------------------------------------------
URLs         0
Headline     0
Body        21
Label        0
dtype: int64
-------------------------------------------------------
0    2137
1    1872
Name: Label, dtype: int64


# Обработка текста



In [8]:
df = df.dropna().reset_index(drop = True)
print(df.isna().sum())

URLs        0
Headline    0
Body        0
Label       0
dtype: int64


In [9]:
from string import punctuation
import re

def remove_punct(text):
    # удаление пунктуации в тексте
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ',
             43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ',
             63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

def txt_prep(df):
    # функция приводит весь текст к нижнему регистру
    # удаляет пунктуацию
    df['Body_01'] = df['Body']
    df['Body_01'] = df['Body_01'].str.lower()
    # остаётся вопрос - нужно ли убирать 's или 'll - эти сокращения явно признак фейковости,
    # после чистки пунктуации там останется s - скорее всего проверка пунктуации определит это как ошибку
    df['Body_01'] = df['Body_01'].map(lambda x: remove_punct(x))

    signs = ['?', ',', ':', '!', '.', ';', '«', '»', '(', ')', '*', '+', '-', '^', '|', '=', "’", "‘",  '”', '—', '@', '>', '<', '–', '&', '%', '•', '/', '“']
    for punct_sign in signs:
        df['Body_01'] = df['Body_01'].str.replace(punct_sign, ' ')

    df['Body_01'] = df['Body_01'].str.replace('\n', '')
    df['Body_01'] = df['Body_01'].str.replace(r"\d+", "", flags=re.UNICODE)
    df['Body_01'] = df['Body_01'].str.replace("  " , " ")
    df['Body_01'] = df['Body_01'].str.replace("  " , " ")

    return df

df = txt_prep(df)

In [10]:
df.head()

,URLs,Headline,Body,Label,Body_01
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,image copyright getty imageson sunday morning ...
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,london reuters last flag flying a comedy dram...
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,the feud broke into public view last week when...
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,mexico city reuters egypt s cheiron holdings l...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,country singer jason aldean who was performing...


## Проверка и исправление на ошибки

In [11]:
!sudo apt-get install swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [12]:
!sudo pip install jamspell

In [13]:
!wget https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz

--2021-12-02 22:15:25--  https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bakwc/JamSpell-models/master/en.tar.gz [following]
--2021-12-02 22:15:25--  https://raw.githubusercontent.com/bakwc/JamSpell-models/master/en.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36611828 (35M) [application/octet-stream]
Saving to: ‘en.tar.gz.1’

en.tar.gz.1         100%[===================>]  34.92M  91.0MB/s    in 0.4s    

2021-12-02 22:15:26 (91.0 MB/s) - ‘en.tar.gz.1’ saved [36611828/36611828]



In [14]:
!tar -xvf en.tar.gz

en.bin


In [15]:
from jamspell import TSpellCorrector

In [16]:
# print(df.loc[5, 'Body_01'])
# print('-------------------------------------------------------')
# print(corrector.FixFragment(df.loc[5, 'Body_01']))

In [17]:
def spellCorrectionofText(df):
    corrector = TSpellCorrector()
    assert corrector.LoadLangModel('en.bin')

    correctedTexts = []
    numberOfSpellMistakes = []

    for index in range(len(df['Body_01'])):

        # text = df['Body_01'][index]
        split_text = df['Body_01'][index].split(' ')

        new_text = ""
        numberMistakes = 0

        for word in split_text:
            corrected_word = corrector.FixFragment(word)
            if word != corrected_word:
                numberMistakes += 1
            new_text += corrected_word + " "
            
        numberOfSpellMistakes.append(numberMistakes)
        correctedTexts.append(new_text)

    df['Body_02'] = correctedTexts
    df['Number of spell mistakes'] = numberOfSpellMistakes

spellCorrectionofText(df)


In [18]:
df

,URLs,Headline,Body,Label,Body_01,Body_02,Number of spell mistakes
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,image copyright getty imageson sunday morning ...,image copyright getty images sunday morning do...,4
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,london reuters last flag flying a comedy dram...,london reuters last flag flying a comedy dram...,0
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,the feud broke into public view last week when...,the feud broke into public view last week when...,3
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,mexico city reuters egypt s cheiron holdings l...,mexico city reuters egypt s chevron holdings l...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,country singer jason aldean who was performing...,country singer jason aldean who was performing...,0
...,...,...,...,...,...,...,...
3983,http://beforeitsnews.com/u-s-politics/2017/10/...,CNN and Globalist Exposed - Steve Quayle and A...,"Vietnam Is in Great Danger, You Must Publish a...",0,vietnam is in great danger you must publish an...,vietnam is in great danger you must publish an...,0
3984,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0,trends to watch of readers think this story is...,trends to watch of readers think this story is...,8
3985,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,trump jr is soon to give a minute speech for ...,trump jr is soon to give a minute speech for ...,0
3986,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,1,shanghai reuters china said it plans to accept...,shanghai reuters china said it plans to accept...,1


# Лемматизация

In [19]:
import sys
!{sys.executable} -m pip install spacy

!{sys.executable} -m spacy download en

     |████████████████████████████████| 12.0 MB 4.8 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [20]:
import spacy

In [21]:
def lemmatizeText(df):

    lemmatizer = spacy.load('en')

    df['Body_03'] = df['Body_02']

    for index in range(len(df['Body_02'])):
        doc = lemmatizer(df['Body_03'][index])
        sentence = " ".join([token.lemma_ for token in doc])

        df['Body_03'][index] = sentence

    df['Body_03'] = df['Body_03'].str.replace("-PRON-", '')
    df['Body_03'] = df['Body_03'].str.replace("s", '')
    df['Body_03'] = df['Body_03'].str.replace("  ", '')

lemmatizeText(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [22]:
print(df['Body_03'][0])
print('-----------------------------------------')
print(df['Body_02'][0])

image copyright getty image unday morning donald trump go off on a twitt tirade againt a member ofown party thi inin t exactly huge new far from the firt time the preident have turnrhetorical cannon onown rank thi time howeverattack be particularly biting and peronaleentially call tenneee enator bob corker the chair of the powerful enate foreign relation committee a coward for not run for re electionay mr corker beg for the preidentendorement whichrefue to givewrongly claim that mr corkerupport of the iranian nuclear agreement beonly political accomplihment unlike ome ofcolleague mr corker free from have to worry aboutimmediate political future didn t holdtongue kip twitter pot by enbobcorker a hame the white houe have become an adult day care center omeone obviouly mihift thi morning enator bob corker enbobcorker october reportthat wan t the end ofthoughthen peak with the new york time and really let the preident havehere be four choice quote from the tenneee enatorinterview with the 

# Удаление стоп-слов

In [23]:
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
english_stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
def removeStopWords(df):
    df['Body_04'] = df['Body_03']

    for stop_word in english_stopwords:
        stopword = r"\b" + stop_word + r"\b"
        df['Body_04'] = df['Body_04'].str.replace(stopword, '')

    df['Body_04'] = df['Body_04'].str.replace('  ', ' ')
    df['Body_04'] = df['Body_04'].str.replace('  ', ' ')

removeStopWords(df)

In [25]:
print(df['Body_03'][0])
print('-----------------------------------------')
print(df['Body_04'][0])

image copyright getty image unday morning donald trump go off on a twitt tirade againt a member ofown party thi inin t exactly huge new far from the firt time the preident have turnrhetorical cannon onown rank thi time howeverattack be particularly biting and peronaleentially call tenneee enator bob corker the chair of the powerful enate foreign relation committee a coward for not run for re electionay mr corker beg for the preidentendorement whichrefue to givewrongly claim that mr corkerupport of the iranian nuclear agreement beonly political accomplihment unlike ome ofcolleague mr corker free from have to worry aboutimmediate political future didn t holdtongue kip twitter pot by enbobcorker a hame the white houe have become an adult day care center omeone obviouly mihift thi morning enator bob corker enbobcorker october reportthat wan t the end ofthoughthen peak with the new york time and really let the preident havehere be four choice quote from the tenneee enatorinterview with the 

In [26]:
df

,URLs,Headline,Body,Label,Body_01,Body_02,Number of spell mistakes,Body_03,Body_04
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,image copyright getty imageson sunday morning ...,image copyright getty images sunday morning do...,4,image copyright getty image unday morning dona...,image copyright getty image unday morning dona...
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,london reuters last flag flying a comedy dram...,london reuters last flag flying a comedy dram...,0,london reuter lat flag fly a comedy drama abou...,london reuter lat flag fly comedy drama vietna...
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,the feud broke into public view last week when...,the feud broke into public view last week when...,3,the feud break into public view lat week when ...,feud break public view lat week mr corker ay ...
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,mexico city reuters egypt s cheiron holdings l...,mexico city reuters egypt s chevron holdings l...,1,mexico city reuter egyptchevron holding limite...,mexico city reuter egyptchevron holding limite...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,country singer jason aldean who was performing...,country singer jason aldean who was performing...,0,country inger jaon aldean who be perform a the...,country inger jaon aldean perform la vega hoot...
...,...,...,...,...,...,...,...,...,...
3983,http://beforeitsnews.com/u-s-politics/2017/10/...,CNN and Globalist Exposed - Steve Quayle and A...,"Vietnam Is in Great Danger, You Must Publish a...",0,vietnam is in great danger you must publish an...,vietnam is in great danger you must publish an...,0,vietnam be in great dangermut publih and tella...,vietnam great dangermut publih tellarmy govern...
3984,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0,trends to watch of readers think this story is...,trends to watch of readers think this story is...,8,trend to watch of reader think thi tory be fac...,trend watch reader think thi tory fact addtwo ...
3985,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,trump jr is soon to give a minute speech for ...,trump jr is soon to give a minute speech for ...,0,trump jr be oon to give a minute peech for of ...,trump jr oon give minute peech reader think th...
3986,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,1,shanghai reuters china said it plans to accept...,shanghai reuters china said it plans to accept...,1,hanghai reuter china ayplan to accept datum fr...,hanghai reuter china ayplan accept datum overe...


# Тональность и субъективность

In [30]:
from textblob import TextBlob

In [33]:
def polarityOfText(df):

    polarity_text = []

    for text in df['Body_04']:
        analysisPol = TextBlob(text).polarity
        polarity_text.append(analysisPol)

    df['Polarity'] = polarity_text

def subjectivityOfText(df):
    subjectivity_text = []

    for text in df['Body_04']:
        analysisSub = TextBlob(text).subjectivity
        subjectivity_text.append(analysisSub)

    df['Subjectivity'] = subjectivity_text


polarityOfText(df)
subjectivityOfText(df)

In [34]:
df

,URLs,Headline,Body,Label,Body_01,Body_02,Number of spell mistakes,Body_03,Body_04,Polarity,Subjectivity
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,image copyright getty imageson sunday morning ...,image copyright getty images sunday morning do...,4,image copyright getty image unday morning dona...,image copyright getty image unday morning dona...,0.049941,0.422146
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,london reuters last flag flying a comedy dram...,london reuters last flag flying a comedy dram...,0,london reuter lat flag fly a comedy drama abou...,london reuter lat flag fly comedy drama vietna...,0.076455,0.460599
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,the feud broke into public view last week when...,the feud broke into public view last week when...,3,the feud break into public view lat week when ...,feud break public view lat week mr corker ay ...,0.135501,0.408650
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,mexico city reuters egypt s cheiron holdings l...,mexico city reuters egypt s chevron holdings l...,1,mexico city reuter egyptchevron holding limite...,mexico city reuter egyptchevron holding limite...,0.119898,0.282653
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,country singer jason aldean who was performing...,country singer jason aldean who was performing...,0,country inger jaon aldean who be perform a the...,country inger jaon aldean perform la vega hoot...,0.043561,0.408333
...,...,...,...,...,...,...,...,...,...,...,...
3983,http://beforeitsnews.com/u-s-politics/2017/10/...,CNN and Globalist Exposed - Steve Quayle and A...,"Vietnam Is in Great Danger, You Must Publish a...",0,vietnam is in great danger you must publish an...,vietnam is in great danger you must publish an...,0,vietnam be in great dangermut publih and tella...,vietnam great dangermut publih tellarmy govern...,0.800000,0.750000
3984,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0,trends to watch of readers think this story is...,trends to watch of readers think this story is...,8,trend to watch of reader think thi tory be fac...,trend watch reader think thi tory fact addtwo ...,0.321465,0.516919
3985,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,trump jr is soon to give a minute speech for ...,trump jr is soon to give a minute speech for ...,0,trump jr be oon to give a minute peech for of ...,trump jr oon give minute peech reader think th...,0.090909,0.303030
3986,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,1,shanghai reuters china said it plans to accept...,shanghai reuters china said it plans to accept...,1,hanghai reuter china ayplan to accept datum fr...,hanghai reuter china ayplan accept datum overe...,0.095141,0.309675


# Векторизация

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
# tfidf = TfidfVectorizer(encoding='utf-8',
#                         ngram_range=(1,3),
#                         stop_words=None,
#                         lowercase=False,
#                         norm='l2',
#                         sublinear_tf=True)
                        
# features = tfidf.fit_transform(df['Body_04']).toarray() # fit()-обучение transfrom()-применение
# feature_names = tfidf.get_feature_names()

# df_features = pd.DataFrame(features, columns=feature_names)

# # features_test = tfidf.transform(X_test).toarray()
# # labels_test = y_test
# # print(features_test.shape)

In [29]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()

# # text_parsed = cv.fit_transform(text_parsed)